## Social media viewer

In [3]:
from ipyleaflet import Map, Marker, MarkerCluster, basemaps, Popup
from ipywidgets import Output, HTML, widgets
from ipyleaflet import AwesomeIcon


In [4]:
import pandas as pd
df = pd.read_csv("Flickr/Flickr_metadata.csv")
df_sv = pd.read_csv('Street_View/meta_streetview_useful.csv')
#df_sv.head()

In [5]:
class GlobalThing:
    def __init__(self, df, displayed_colnames = None):
        self.marker_info = dict()
        self.df = df
        if displayed_colnames is None:
            displayed_colnames = ["Image"]
        self.displayed_colnames = displayed_colnames
    
    def add_marker_info(self, lat, lon, Image):
        self.marker_info[(lat, lon)] = Image
    
    def get_code(self, lat, lon):
        return self.marker_info[(lat,lon)]

    def data_for_hydroid(self, ident):
        df_sub = self.bore_data.loc[df["Image"] == ident]
        return df_sub[self.displayed_colnames]

    def register_geolocations(self, geoloc):
        for index, row in geoloc.iterrows():
            self.add_marker_info(row.Lat, row.Lon)
            
globalthing = GlobalThing(df, displayed_colnames=['Image'])


In [8]:
def plot_map(geoloc, geoloc_sv, click_handler):
    """
    Plot the markers for each borehole, and register a custom click_handler
    """
    mean_lat = geoloc_sv.Lat.mean()
    mean_lng = geoloc_sv.Lon.mean()
    center = [-14, -40]
    # create the map
    m = Map(basemap=basemaps.Stamen.Terrain,center=center, zoom=4, scroll_wheel_zoom=True)
    m.layout.height = '1200px'
    # show trace
    markers_sv = []
    markers = []
    long_string_to_enlarge_the_image = "..........."*7
    # long_string_to_enlarge_the_image = "&nbsp; "#*7
    for index, row,  in geoloc.iterrows():
        message = HTML()
        image_str = str(row.Image)
        image_path = f"Flickr/{image_str}.jpg"
       
        image_html_path = f'<img src="{image_path}" width="500" height ="500"></img>{long_string_to_enlarge_the_image}'
        message = HTML(image_html_path)
        marker = Marker(location=(row.Lat, row.Lon),draggable=False)
        # marker.on_click(click_handler)
        marker.popup = Popup( child=message, close_button=True, auto_close=False, close_on_escape_key=True, 
             max_width=600, max_height=600)
        markers.append(marker)
    marker_cluster = MarkerCluster(
        markers=markers
    )
    for index, row,  in geoloc_sv.iterrows():
        message = HTML()
        image_str = str(row.Panoid)
        image_path = f"Street_View/{image_str}.jpg"
        
        image_html_path = f'<img src="{image_path} width="500" height ="500"></img>{long_string_to_enlarge_the_image}'
        message = HTML(image_html_path)
        marker = Marker(location=(row.Lat, row.Lon), draggable = False, icon = AwesomeIcon(name='globe', marker_color='red') )
        # marker.on_click(click_handler)
        marker.popup = Popup( child=message, close_button=True, auto_close=False, close_on_escape_key=True, 
             max_width=900, max_height=600)
        markers_sv.append(marker)
    marker_cluster_sv = MarkerCluster(
        markers=markers_sv
    )
    # not sure whether we could register once instead of each marker:
    # marker_cluster.on_click(click_handler)
    m.add_layer(marker_cluster);
    m.add_layer(marker_cluster_sv)
    # m.add_control(FullScreenControl())
    return m

# If printing a data frame straight to an output widget
def raw_print(out, ident):
    bore_data = globalthing.data_for_hydroid(ident)
    out.clear_output()
    with out:
        print(ident)        
        print(bore_data)
        
def click_handler_rawprint(**kwargs):
    blah = dict(**kwargs)
    xy = blah['coordinates']
    print(xy)
    ident = globalthing.get_code(xy[0], xy[1])
    raw_print(out, ident)

    # to display using an ipysheet
def mk_sheet(d):
    return ipysheet.pandas_loader.from_dataframe(d)

def upate_display_df(ident):
    bore_data = globalthing.data_for_hydroid(ident)
    out.clear_output()
    with out:
        display(mk_sheet(bore_data))        

def click_handler_ipysheet(**kwargs):
    blah = dict(**kwargs)
    xy = blah['coordinates']
    ident = globalthing.get_code(xy[0], xy[1])
    upate_display_df(ident)


In [9]:
# m = Map(center=center, zoom=2, close_popup_on_click=False,scroll_wheel_zoom=True)
# marker = Marker(location=(-16.234196, -48.662695))
# m.add_layer(marker)
# image_path = "11756320755.jpg"
# long_string_to_enlarge_the_image = "&nbsp; "*300
# image_html_path = f'<img src="{image_path}"></img>{long_string_to_enlarge_the_image}'
# message1 = HTML(image_html_path)
# Popup with a given location on the map:
# popup = Popup( location=center, child=message1, close_button=True, auto_close=False, close_on_escape_key=True, 
#              max_width=99999999, max_height=9999999)
# Popup associated to a layer
# marker.popup = message1

plot_map(df, df_sv, click_handler_ipysheet)


Map(center=[-14, -40], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…